# Máquinas de Vectores de Soporte

In [1]:
import numpy as np
import pandas as pd

## Importando los datos

In [3]:
mammographic_standard = pd.read_csv('../processed/mammographic_masses_standard.csv')
mammographic_standard.head(10)

,Age,Shape,Margin,Density,Severity
0,0.765804,0.174460,1.395631,0.240313,1
1,0.151666,0.979883,1.395631,0.240313,1
2,-1.895458,-1.436386,-1.158927,0.240313,0
3,0.083429,-1.436386,1.395631,0.240313,1
4,1.379941,-1.436386,0.756992,0.240313,1
5,-0.940133,-0.630963,-1.158927,0.240313,1
6,-1.349558,0.174460,-1.158927,-2.612545,0
7,0.288141,-0.630963,-1.158927,-2.612545,0
8,-0.121284,-1.436386,-1.158927,0.240313,0
9,-0.257759,0.174460,0.756992,0.240313,0


## Datos de entrenamiento y prueba

In [7]:
df_feat = mammographic_standard.drop('Severity', axis = 1)
df_feat.head()

,Age,Shape,Margin,Density
0,0.765804,0.174460,1.395631,0.240313
1,0.151666,0.979883,1.395631,0.240313
2,-1.895458,-1.436386,-1.158927,0.240313
3,0.083429,-1.436386,1.395631,0.240313
4,1.379941,-1.436386,0.756992,0.240313


In [8]:
df_target = mammographic_standard['Severity'].to_frame()
df_target.head()

,Severity
0,1
1,1
2,0
3,1
4,1


In [4]:
from sklearn.model_selection import train_test_split

In [9]:
X_train, X_test, y_train, y_test = train_test_split(df_feat, np.ravel(df_target), test_size=0.30)

## Entrenamiento de la Máquina de Soporte

In [11]:
from sklearn.svm import SVC
scv_mm = SVC(gamma = 'scale')
scv_mm.fit(X_train,y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

## Predicciones y Evaluaciones

In [12]:
y_predict = scv_mm.predict(X_test)

In [13]:
from sklearn.metrics import classification_report,confusion_matrix

### Matriz de confución

In [14]:
print(confusion_matrix(y_test,y_predict))

[[ 90  45]
 [ 11 104]]


### Reporte de clasificación

In [16]:
print(classification_report(y_test,y_predict))

              precision    recall  f1-score   support

           0       0.89      0.67      0.76       135
           1       0.70      0.90      0.79       115

   micro avg       0.78      0.78      0.78       250
   macro avg       0.79      0.79      0.78       250
weighted avg       0.80      0.78      0.77       250



## Búsqueda de hiperparámetros con Gridsearch

In [59]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    'C': [0.1, 1, 10, 100, 1000], 
    'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
    'coef0': [1, 2, 3],
    'kernel': ['rbf', 'sigmoid']
} 
grid = GridSearchCV(SVC(),param_grid,refit=True,cv=5, verbose=3)
grid.fit(X_train,y_train)

Fitting 5 folds for each of 150 candidates, totalling 750 fits
[CV] C=0.1, coef0=1, gamma=1, kernel=rbf .............................
[CV]  C=0.1, coef0=1, gamma=1, kernel=rbf, score=0.8034188034188035, total=   0.0s
[CV] C=0.1, coef0=1, gamma=1, kernel=rbf .............................
[CV]  C=0.1, coef0=1, gamma=1, kernel=rbf, score=0.7692307692307693, total=   0.0s
[CV] C=0.1, coef0=1, gamma=1, kernel=rbf .............................
[CV]  C=0.1, coef0=1, gamma=1, kernel=rbf, score=0.7264957264957265, total=   0.0s
[CV] C=0.1, coef0=1, gamma=1, kernel=rbf .............................
[CV]  C=0.1, coef0=1, gamma=1, kernel=rbf, score=0.808695652173913, total=   0.0s
[CV] C=0.1, coef0=1, gamma=1, kernel=rbf .............................
[CV]  C=0.1, coef0=1, gamma=1, kernel=rbf, score=0.8173913043478261, total=   0.0s
[CV] C=0.1, coef0=1, gamma=1, kernel=sigmoid .........................
[CV]  C=0.1, coef0=1, gamma=1, kernel=sigmoid, score=0.7264957264957265, total=   0.0s
[CV] C=0.1

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


[CV]  C=0.1, coef0=1, gamma=0.1, kernel=sigmoid, score=0.7264957264957265, total=   0.0s
[CV] C=0.1, coef0=1, gamma=0.1, kernel=sigmoid .......................
[CV]  C=0.1, coef0=1, gamma=0.1, kernel=sigmoid, score=0.8347826086956521, total=   0.0s
[CV] C=0.1, coef0=1, gamma=0.1, kernel=sigmoid .......................
[CV]  C=0.1, coef0=1, gamma=0.1, kernel=sigmoid, score=0.8608695652173913, total=   0.0s
[CV] C=0.1, coef0=1, gamma=0.01, kernel=rbf ..........................
[CV]  C=0.1, coef0=1, gamma=0.01, kernel=rbf, score=0.7948717948717948, total=   0.0s
[CV] C=0.1, coef0=1, gamma=0.01, kernel=rbf ..........................
[CV]  C=0.1, coef0=1, gamma=0.01, kernel=rbf, score=0.811965811965812, total=   0.0s
[CV] C=0.1, coef0=1, gamma=0.01, kernel=rbf ..........................
[CV]  C=0.1, coef0=1, gamma=0.01, kernel=rbf, score=0.7264957264957265, total=   0.0s
[CV] C=0.1, coef0=1, gamma=0.01, kernel=rbf ..........................
[CV]  C=0.1, coef0=1, gamma=0.01, kernel=rbf, scor

[Parallel(n_jobs=1)]: Done 750 out of 750 | elapsed:   19.8s finished
C:\conda\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'C': [0.1, 1, 10, 100, 1000], 'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 'coef0': [1, 2, 3], 'kernel': ['rbf', 'sigmoid']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

In [60]:
yg_predict = grid.predict(X_test)

### Mejores parametros

In [61]:
grid.best_params_

{'C': 0.1, 'coef0': 1, 'gamma': 0.1, 'kernel': 'sigmoid'}

### Mejor estimador

In [62]:
grid.best_estimator_

SVC(C=0.1, cache_size=200, class_weight=None, coef0=1,
  decision_function_shape='ovr', degree=3, gamma=0.1, kernel='sigmoid',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

### Matriz de confución

In [63]:
print(confusion_matrix(y_test,yg_predict))

[[ 92  43]
 [ 12 103]]


### Reporte de clasificación

In [64]:
print(classification_report(y_test,yg_predict))

              precision    recall  f1-score   support

           0       0.88      0.68      0.77       135
           1       0.71      0.90      0.79       115

   micro avg       0.78      0.78      0.78       250
   macro avg       0.80      0.79      0.78       250
weighted avg       0.80      0.78      0.78       250

